In [11]:
db = int(input('Database ID (2 for 4 chamber and 17 for short axis): '))
basedir = input('Base directory (e.g. D:/ML_data/PAH): ')
scale = int(input('Scale (16, 8, 4, or -1): '))
mask_id = int(input('Mask ID: '))
level = int(input('Preprocessing level: '))

Database ID (2 for 4 chamber and 17 for short axis): 2
Base directory (e.g. D:/ML_data/PAH): D:/ML_data/PAH 
Scale (16, 8, 4, or -1):16
Mask ID: 5
Preprocessing level: 1


### Step 0: Converting .mat file to .npy (ndarray) file

In [1]:
import os
import h5py
import numpy as np
import pandas as pd


def mat2npy(basedir, db):  
    fname = 'PAH1DB%s.mat' % db
    print('Converting %s to ndarray' % fname)
    data_path = os.path.join(basedir, fname)
    f = h5py.File(data_path, 'r')

    data = f['data'][()].transpose()
    out_path = os.path.join(basedir, 'PAH1DB%s.npy' % db)
    np.save(out_path, data)
    # data_ = torch.from_numpy(data).to_sparse()
    # out_path = os.path.join(basedir, 'RegPAH1DB%s.hdf5' % db)
    # f = h5py.File(out_path, "w")
    # dest = f.create_dataset()

    labels = f['labels'][()].reshape(-1)
    # max_dist = f['maxDists'][()].reshape(-1)
    # df = pd.DataFrame(data={'Label': labels, 'Max dist': max_dist})
    df = pd.DataFrame(data={'Label': labels})
    csv_path = os.path.join(basedir, 'info_DB%s.csv' % db)
    df.to_csv(csv_path)
    print('Completed!')

In [ ]:
mat2npy(basedir, db)

## Step 1: Registration

In [2]:
import sys
sys.path.append('..')
from kale.prepdata.prep_cmr import regMRI

def proc_reg(basedir, db):
    print('Performing registration...')
    data_path = os.path.join(basedir, 'PAH1DB%s.npy' % db)
    data = np.load(data_path)
    reg_fpath = os.path.join(basedir, 'regMRI4ChFull.xlsx')
    # reg_fpath = os.path.join('C:\GitHub\PAH\MatData/regNewFold1.xlsx')
    if db == 2:
        sheet_name = '4ch'
        # sheet_name = 0
        col_names = ['ID', 'Group', 'mitral ann X', 'mitral ann Y',
                     'LVEDV apex X', 'LVEDV apex Y', 'Spinal X', 'Spinal Y']
    elif db == 17:
        sheet_name = 'SA'
        col_names = ['ID', 'Group', 'inf insertion point X', 'insertion point Y',
                     'sup insertion point X', 'sup insertion point Y', 'RV inf X', 'RV inf Y']

    reg_file = pd.read_excel(reg_fpath, sheet_name=sheet_name, index_col='ID', usecols=col_names)
    data_reg, max_dist = regMRI(data, reg_file)
    out_path = os.path.join(basedir, 'RegPAH1DB%s.npy' % db)
    np.save(out_path, data_reg)

    info_file = os.path.join(basedir, 'info_DB%s.csv' % db)
    if os.path.exists(info_file):
        info_df = pd.read_csv(info_file, index_col=0)
    else:
        info_df = pd.DataFrame(data={'Label': reg_file['Group'].values})
    info_df['ID'] = reg_file['ID']
    info_df['Max Dist'] = max_dist
    info_df.to_csv(info_file, columns=['ID', 'Label', 'Max Dist'], index=False)

    print('Registration Completed')

In [ ]:
proc_reg(basedir, db)

## Step 2: Rescaling

In [3]:
import sys
sys.path.append('..')
from kale.prepdata.prep_cmr import rescale_cmr

def proc_rescale(basedir, db, scale=-1):
    data_path = os.path.join(basedir, 'RegPAH1DB%s.npy' % db)
    data = np.load(data_path)
    out_dir = os.path.join(basedir, 'DB%s' % db)
    print('Rescaling data ...')
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)

    if scale == -1:
        for scale_ in [16, 8, 4]:
            print('Scale: 1/%s' % scale_)
            data_ = rescale_cmr(data, scale=scale_)
            out_path = os.path.join(out_dir, 'NoPrs%sDB%s.npy' % (scale_, db))
            np.save(out_path, data_)
    else:
        print('Scale: 1/%s' % scale)
        data_ = rescale_cmr(data, scale=scale)
        out_path = os.path.join(out_dir, 'NoPrs%sDB%s.npy' % (scale, db))
        np.save(out_path, data_)

    print('Completed!')

In [ ]:
proc_rescale(basedir, db, scale)

## Step 3: Preprocessing

In [ ]:
import sys
sys.path.append('..')
from kale.prepdata.prep_cmr import cmr_proc

if scale == -1:
    for scale_ in [16, 8, 4]:
        cmr_proc(basedir, db, scale_, mask_id, level, save_data=True)
else:
    cmr_proc(basedir, db, scale, mask_id, level, save_data=True)


# Classification

In [9]:
import sys
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV


default_grid = [
    {'select__estimator__C': np.logspace(-2, 2, 5)},
    {'clf__C': np.logspace(-3, 2, 6), 'clf__kernel': ['linear']},
    {'clf__C': np.logspace(-3, 2, 6), 'clf__gamma': np.logspace(-4, -1, 3),
     'clf__kernel': ['rbf']},
    ]

# clf = Pipeline([
#   ('feature_selection', SelectFromModel(LinearSVC(penalty="l1"))),
#   ('classification', RandomForestClassifier())
# ])


class _Classifier(BaseEstimator, TransformerMixin):
    def __init__(self, clf='SVC', param_grid=default_grid, cv=None, n_split=10, test_size=0.2, n_jobs=1):
        if clf == 'SVC':
            # _clf = Pipeline([('select', SelectFromModel(LinearSVC(penalty='l1', loss='hinge'))),
            _clf = Pipeline([('select', SelectFromModel(estimator=LogisticRegression(penalty='l1', solver='liblinear'))),
                             ('clf', SVC(max_iter=10000))])
        elif clf == 'LR':
            _clf = Pipeline([('select', SelectFromModel(Lasso())),
                             ('clf', LogisticRegression(max_iter=10000))])
        elif clf == 'Ridge':
            _clf = Pipeline([('select', SelectFromModel(Lasso())),
                             ('clf', RidgeClassifier(max_iter=10000))])
        else:
            print('Invalid Classifier')
            sys.exit()
        
        print(param_grid)
        if cv is None:
            cv = StratifiedShuffleSplit(n_splits=n_split, test_size=test_size,
                                        train_size=1 - test_size, random_state=144)
        self.search = GridSearchCV(_clf, param_grid, n_jobs=n_jobs, cv=cv, iid=False)

    def fit(self, X, y):
        self.search.fit(X, y)
        self.clf = self.search.best_estimator_
        self.clf.fit(X, y)

        return self

    def predict(self, X):
        return self.clf.predict(X)

In [10]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from tensorly.base import fold, unfold


def label_binarizer(y):
    y_ = np.zeros(y.shape)
    y_[np.where(y != 0)] = 1
    return y_


def evaluate_(X, y, kfold=10, random_state=144, return_auc=True):
    skf = StratifiedKFold(n_splits=kfold, random_state=random_state)
    res = {'fold_accs': [], 'fold_aucs': [], 'acc': None,'auc': None}
    y_pred = np.zeros(y.shape)
    y_dec = np.zeros(y.shape)
    for train, test in skf.split(X, y):
        clf = _Classifier()
        clf.fit(X[train], y[train])
        y_pred[test] = clf.predict(X[test])
        res['fold_accs'].append(accuracy_score(y[test], y_pred[test]))
        if return_auc:
            y_dec[test] = clf.decision_function(X[test])
            res['fold_aucs'].append(roc_auc_score(y[test], y_dec[test]))
    res['acc'] = accuracy_score(y, y_pred)
    if return_auc:
        res['auc'] = roc_auc_score(y, y_dec)

    return res

In [ ]:
from kale.embed.mpca import MPCA

print('Main Experiemnts for Scale: 1/%s, Mask ID: %s, Processing level: %s' % (scale, mask_id, level))
data_path = '%s/DB%s/PrepData' % (basedir, db)
fname = 'PrS%sM%sL%sDB%s.npy' % (scale, mask_id, level, db)
X = np.load(os.path.join(data_path, fname))
info_df = pd.read_csv(os.path.join(basedir, 'info_DB%s.csv' % db))
y = info_df['Label'].values
y_ = label_binarizer(y)

# Peform MPCA dimension reduction
mpca = MPCA()
mpca.fit(X)
Xmpc = mpca.transform(X)
X_ = unfold(Xmpc, mode=-1).real

# Evaluating 
res = evaluate_(X_, y_)

print('Accuracy:', res['acc'], 'AUC:', res['auc'])
